# Santander Customer Transaction Prediction Analysis:

## Comparing Using only Features Where Mean or SD of Positive-Target Subset Is Significantly Different from Original Dataset

In [ ]:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np

from scipy.stats import ttest_ind, levene

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('../input/train.csv')

In [ ]:
df.head()

### Checking for dataset issues:

In [ ]:
#data types by column
df.dtypes.value_counts()

In [ ]:
#mising values
df.isnull().sum().sum()

In [ ]:
# Number of 1s in target, total rows, and percent of 1s in target column:
positives = df.target.sum()
total = len(df.target)
print('positives:',positives,'total:',total,'percent:','{}%'.format(positives/total*100))

## Hypothesis testing features in relation to their target subset (by mean and standard deviation):

In [ ]:
pvalue_index_list = []

for i in range(200):
    data1 = df['var_'+str(i)].values
        
    data2 = df[df['target']==1]['var_'+str(i)].values
        
    test_stat1, p_value_mean = ttest_ind(data1,data2)
    
    test_stat2, p_value_sd = levene(data1,data2)
    
    pvalue_index_list.append([i, p_value_mean, p_value_sd])

In [ ]:
df_pvalues = pd.DataFrame(pvalue_index_list, columns=['column_name',
                                                      'p_value_mean',
                                                      'p_value_sd'])
df_pvalues.set_index('column_name',inplace=True)

In [ ]:
df_pvalues.head()

In [ ]:
#Gives indication of number of significant values (by p-value)
#Then we will remove everything not below 1.25e-4 p-value threshold
histogram = df_pvalues.hist(bins=20)

In [ ]:
df_pvalues.p_value_mean.describe()

In [ ]:
df_pvalues.p_value_sd.describe()

### Separating out statistically different features:

In [ ]:
#The 1.25e-4 threshold was generated by taking the critical value .05 and
#and dividing it by 400 (200 features tested for two characteristics)
significant_columns1 = df_pvalues[df_pvalues['p_value_mean']<=1.25e-4].index

significant_columns2 = df_pvalues[df_pvalues['p_value_sd']<=1.25e-4].index

sig_col = significant_columns1.union(significant_columns2)

In [ ]:
columns_list = ['var_'+str(i) for i in sig_col]

df_sig = df[['target']+columns_list]

In [ ]:
df_sig.head()

## Separating training and test sets:

In [ ]:
#Pruned values
X_all1 = df_sig.drop(['target'],axis=1)
y_all1 = df_sig['target']

#All values
X_all2 = df.drop(['target','ID_code'],axis=1)
y_all2 = df['target']

## Machine learning:

In [ ]:
#classifiers
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression

import xgboost as xgb

#for function below
from sklearn.model_selection import StratifiedKFold
from time import time
from sklearn.metrics import make_scorer,confusion_matrix,accuracy_score,\
    precision_score,recall_score,f1_score,roc_auc_score,matthews_corrcoef

### Balancing dataset target:

In [ ]:
#for xgboost scale_pos_weight
xgb_weight = (total-positives)/positives

xgb_weight

In [ ]:
clf_A = LogisticRegression(random_state=0,class_weight='balanced')

clf_B = RandomForestClassifier(random_state=0,max_depth=3,class_weight='balanced')

clf_C = xgb.XGBClassifier(random_state=0,max_depth=3,scale_pos_weight=xgb_weight,
                          subsample=.5,sample_type='weighted')

### K-Fold cross-validation function:

In [ ]:
average_orig_auc=[]
average_prune_auc=[]

def metrics_function(target,pred):
    return accuracy_score(target, pred),precision_score(target, pred),\
        recall_score(target, pred),f1_score(target, pred),\
        roc_auc_score(target, pred),matthews_corrcoef(target, pred)

def FOLD_TEST(clf,X_all,y_all,folds_num,row_factor,orig_or_prune):
    start=time()
    
    KFLD=StratifiedKFold(n_splits=folds_num,random_state=662,shuffle=True)
    print ('{}:'.format(clf.__class__.__name__),'\n')
    
    acc_list_train=[]
    acc_list_test=[]
    prc_list_train=[]
    prc_list_test=[]
    rcal_list_train=[]
    rcal_list_test=[]
    f1_list_train=[]
    f1_list_test=[]
    matt_list_train=[]
    matt_list_test=[]
    AUC_list_train=[]
    AUC_list_test=[]
    
    samp_size=X_all.shape[0]//row_factor
    
    for fold,(train_index,target_index) in enumerate(KFLD.split(X_all[:samp_size],
                                                                y_all[:samp_size])):
        X_train=X_all.iloc[train_index].values
        y_train=y_all.iloc[train_index].values

        X_test=X_all.iloc[target_index].values
        y_test=y_all.iloc[target_index].values

        clf.fit(X_train,y_train)
        y_pred1=clf.predict(X_train)
        y_pred2=clf.predict(X_test)

        train_acc,train_prc,train_rcal,train_f1,train_auc,train_matt=metrics_function(y_train,y_pred1)
        
        test_acc,test_prc,test_rcal,test_f1,test_auc,test_matt=metrics_function(y_test,y_pred2)
        
        acc_list_train.append(train_acc)
        acc_list_test.append(test_acc)
        prc_list_train.append(train_prc)
        prc_list_test.append(test_prc)
        rcal_list_train.append(train_rcal)
        rcal_list_test.append(test_rcal)
        
        f1_list_train.append(train_f1)
        f1_list_test.append(test_f1)
        matt_list_train.append(train_matt)
        matt_list_test.append(test_matt)
        AUC_list_train.append(train_auc)
        AUC_list_test.append(test_auc)
    
    print("Averages:"'\n')
    
    print("Train acc: {}, Test acc: {}".format(np.mean(acc_list_train),
                                               np.mean(acc_list_test)))
    print("Train prc: {}, Test prc: {}".format(np.mean(prc_list_train),
                                               np.mean(prc_list_test)))
    print("Train recall: {}, Test recall: {}".format(np.mean(rcal_list_train),
                                                     np.mean(rcal_list_test)),'\n')
    
    print("Train f1: {}, Test f1: {}".format(np.mean(f1_list_train),
                                             np.mean(f1_list_test)))
    print("Train MattCC: {}, Test MattCC: {}".format(np.mean(matt_list_train),
                                                     np.mean(matt_list_test)))
    print("Train AUC: {}, Test AUC: {}".format(np.mean(AUC_list_train),
                                               np.mean(AUC_list_test)),'\n'*2)
        
    print("Sample Size: {}, Folds Num: {}, Time: {}".format(samp_size,folds_num,
                                                            time()-start))
    
    if orig_or_prune == 'orig':
        average_orig_auc.append(np.mean(AUC_list_test))
    elif orig_or_prune == 'prune':
        average_prune_auc.append(np.mean(AUC_list_test))
    else:
        print("orig_or_prune argument should be either 'orig' or 'prune'")

### Logistic Regression:

In [ ]:
#Pruned data:
FOLD_TEST(clf_A, X_all1, y_all1,5,7,'prune')

In [ ]:
#Original data:
FOLD_TEST(clf_A, X_all2, y_all2,5,7,'orig')

### Random Forest: 

In [ ]:
#Pruned data:
FOLD_TEST(clf_B, X_all1, y_all1,5,7,'prune')

In [ ]:
#Original data:
FOLD_TEST(clf_B, X_all2, y_all2,5,7,'orig')

### XGBoost:

In [ ]:
#Pruned data:
FOLD_TEST(clf_C, X_all1, y_all1,5,7,'prune')

In [ ]:
#Original data:
FOLD_TEST(clf_C, X_all2, y_all2,5,7,'orig')

## Results:

In [ ]:
print("Avg Original AUC: {}, Avg Pruned AUC: {}".format(np.mean(average_orig_auc),
                                                        np.mean(average_prune_auc)))

### There does not appear to be a significant difference between the original and pruned datasets, but optimizing the k-folds, sample size, and classification parameters might change that.

### However, the pruned data did take less time and produced equivalent results.